<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [138]:
import pandas as pd
import psycopg2
import plotly
import plotly.express as px

In [139]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [140]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [141]:
# текст запроса
query_3_1 = f'''SELECT Count(id) cnt_vacancies
                FROM vacancies
'''

In [142]:
# результат запроса
vacansies_df = pd.read_sql_query(query_3_1, connection)
display(vacansies_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\685587696.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [143]:
# текст запроса
query_3_2 = f'''SELECT Count(id) cnt_employers
                FROM employers
'''

In [144]:
# результат запроса
employers_df = pd.read_sql_query(query_3_2, connection)
display(employers_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\3187934135.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_employers
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [145]:
# текст запроса
query_3_3 = f'''SELECT Count(id) cnt_areas
                FROM areas
'''

In [146]:
# результат запроса
areas_df = pd.read_sql_query(query_3_3, connection)
display(areas_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\2913775617.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_areas
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [147]:
# текст запроса
query_3_4 = f'''SELECT Count(id) cnt_industries
                FROM industries
'''

In [148]:
# результат запроса
industries_df = pd.read_sql_query(query_3_4, connection)
display(industries_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\4227789640.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_industries
0,294


***

In [149]:
# выводы по предварительному анализу данных
res_data = f'''Всего в БД имеются сведения о {vacansies_df['cnt_vacancies'].iloc[0]} открытых вакансиях, 
представленных {employers_df['cnt_employers'].iloc[0]} работотаделями.
Предлагаемые вакансии распределены по {areas_df['cnt_areas'].iloc[0]} регоинам 
и представляют {industries_df['cnt_industries'].iloc[0]} сферы деятельности 
'''
print(res_data)

Всего в БД имеются сведения о 49197 открытых вакансиях, 
представленных 23501 работотаделями.
Предлагаемые вакансии распределены по 1362 регоинам 
и представляют 294 сферы деятельности 



# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [150]:
# текст запроса
query_4_1 = f'''SELECT a.name area,
                       Count(v.id) cnt
                FROM areas a
                  LEFT JOIN
                     vacancies v ON v.area_id = a.id
                GROUP BY a.name
                ORDER BY 2 desc
                LIMIT 5
'''

In [151]:
# результат запроса
area_cnt = pd.read_sql_query(query_4_1, connection)
display(area_cnt)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\2389416541.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


In [152]:
query_4_1_all = f'''SELECT a.name area,
                       Count(v.id) cnt_vac
                FROM areas a
                  LEFT JOIN
                     vacancies v ON v.area_id = a.id
                GROUP BY a.name
                HAVING Count(v.id) >=200
                UNION
                SELECT 'Прочие - ' || Count(t1.area) || ' регионов' area,
                        sum(t1.cnt_vac) cnt_vac
                FROM (SELECT a.name area,
                             Count(v.id) cnt_vac
                      FROM areas a
                        LEFT JOIN
                          vacancies v ON v.area_id = a.id
                      GROUP BY a.name
                      HAVING Count(v.id) < 200) t1
                ORDER BY 2 desc
'''
area_vac_df = pd.read_sql_query(query_4_1_all, connection)

fig = px.bar(
            data_frame = area_vac_df,
            x='area',
            y='cnt_vac',
            title='Диаграмма 1.  Количество вакансий по регионам')
fig.update_layout(
      xaxis_title="Регионы",
      yaxis_title="Количество вакансий",
      font=dict(size=12) 
);

fig.show()

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\78871003.py:20: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [153]:
# текст запроса
query_4_2 = f'''SELECT Count(v.id) cnt_vac
                FROM vacancies v
                WHERE v.salary_from is not null OR 
                      v.salary_to is not null
'''

In [154]:
# результат запроса
vac_zp_df = pd.read_sql_query(query_4_2, connection)
print(vac_zp_df)

   cnt_vac
0    24073


C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\461237172.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [155]:
query_4_2_non_zp = f'''SELECT em.name employer,
                              Count(v.id) cnt_vac
                       FROM vacancies v
                            JOIN
                            employers em ON v.employer_id = em.id
                       WHERE Coalesce(v.salary_from, v.salary_to,0) = 0
                       GROUP BY em.name
                       HAVING Count(v.id) > 100
                       UNION
                       SELECT 'Прочие ' || Count(t1.name) || ' работодателей ' employer,
                               sum(t1.cnt_vac) cnt_vac
                       FROM(SELECT em.name,
                                   Count(v.id) cnt_vac
                            FROM vacancies v
                               JOIN
                                 employers em ON v.employer_id = em.id
                            WHERE Coalesce(v.salary_from, v.salary_to,0) = 0
                            GROUP BY em.name
                            HAVING Count(v.id) <= 100) t1
                       ORDER BY 2 DESC
'''

empl_non_zp = pd.read_sql_query(query_4_2_non_zp, connection)

fig = px.pie(data_frame=empl_non_zp,
             values='cnt_vac', 
             names='employer',
             width=900,
             height=550,
             title='Диаграмма 2. Вакансии без Зарплатной вилки')

fig.update_traces(rotation=-70)
fig.show()

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\3977746206.py:23: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [156]:
# текст запроса
query_4_3 = f'''SELECT Round(Avg(v.salary_from)) avg_from,
                       Round(Avg(v.salary_to)) avg_to
                FROM vacancies v
'''

In [157]:
# результат запроса
avg_salar_df = pd.read_sql_query(query_4_3, connection)
display(avg_salar_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\26266335.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,avg_from,avg_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [158]:
# текст запроса
query_4_4 = f'''SELECT v.schedule,
                       v.employment, 
                       Count(v.id) cnt_vac
                FROM vacancies v
                GROUP BY v.schedule,
                         v.employment
                ORDER BY cnt_vac DESC
                LIMIT 3                                    
'''

In [159]:
# результат запроса

shedule_df = pd.read_sql_query(query_4_4, connection)
display(shedule_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\3155760220.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,schedule,employment,cnt_vac
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [160]:
# текст запроса
query_4_5 = f'''SELECT v.experience,
                    Count(v.id) cnt_vac
                FROM vacancies v
                GROUP BY v.experience
                ORDER BY 2
'''

In [161]:
# результат запроса
cnt_experience_df = pd.read_sql_query(query_4_5, connection)

display(cnt_experience_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\3836829975.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,cnt_vac
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [162]:
query_4_6 = f'''SELECT v.experience,
                       coalesce(v.salary_from, v.salary_to,0)+
                       coalesce(v.salary_to, v.salary_from,0)/2 avg_salary
                  FROM vacancies v
                  WHERE coalesce(v.salary_from, v.salary_to,0) > 0
'''
exper_salary_df = pd.read_sql_query(query_4_6, connection)

fig = px.box(
        data_frame=exper_salary_df,
        x='avg_salary',
        y='experience',
        title='Диаграмма 3. Уровень зарплаты в зависимости от опыта работы')
fig.update_layout(
            xaxis_title = 'Зарплата',
            yaxis_title = 'Опыт работы')

fig.show()


C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\2238717342.py:7: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [163]:
# выводы по детальному анализу вакансий
all_vac = vacansies_df['cnt_vacancies'].iloc[0]
sps_top5city = '; '.join(list(area_cnt['area']))
top5city = area_cnt['cnt'].sum()
perc_top5city = round(top5city/all_vac*100)

indicated_salary = vac_zp_df['cnt_vac'].iloc[0]
perc_null_salary = round((all_vac-indicated_salary)/all_vac*100)
avg_salary_from = avg_salar_df['avg_from'].iloc[0]
avg_salary_to = avg_salar_df['avg_to'].iloc[0]

sps_shedule = '; '.join(list(shedule_df['schedule']))
perc_shedule = round(shedule_df['cnt_vac'].sum()/all_vac*100)

sort_experience = cnt_experience_df.sort_values(by='cnt_vac', ascending=False)
list_experience = '; '.join(list(sort_experience['experience'].iloc[0:2]))
perc_experience = round(sort_experience['cnt_vac'].iloc[0:2].sum()/all_vac*100)

res_data = f'''ТОП5 городов составляют наиболее крупные города: {sps_top5city}, 
на их долю приходится {perc_top5city}% от от бщего количества вакансий. 
В целом диаграмма 1, показывает, что наибольшее количество вакансий приходится на крупные города.
Что указывает на прямую зависимость в потребности рабочих рук от количества проживающих в регионе.

В {perc_null_salary}% вакансий не указана зарплатная вилка (необходима пред. обработка данных в части заполнения пропусков). 
При этом диаграмма 2, показывает, чаще всего вакансии без указания Зарплатиной вилки размещают мелкие работодатели, 
в то же время, у крупных работодателей доля вакансий размещенных без зарплатной вилки составляет не более 2 процентов.

Для тех вакансий где указано хотя бы одно значение зарплатной вилки, 
средние показатели составляют от {avg_salary_from} руб. до {avg_salary_to} руб.
Диаграмма 3 показывает, что опыт работы напрямую влияет на уровень, предлагаемой, заработной платы

Наиболее популярными видами рабочего графика при условии полной занятости, являются: {sps_shedule},
на их долю приходится {perc_shedule}% от всех вакансий.
В {perc_experience}% вакансий, к опыту работы соискателей выдвигаются требования: {list_experience}
'''
print(res_data)


ТОП5 городов составляют наиболее крупные города: Москва; Санкт-Петербург; Минск; Новосибирск; Алматы, 
на их долю приходится 29% от от бщего количества вакансий. 
В целом диаграмма 1, показывает, что наибольшее количество вакансий приходится на крупные города.
Что указывает на прямую зависимость в потребности рабочих рук от количества проживающих в регионе.

В 51% вакансий не указана зарплатная вилка (необходима пред. обработка данных в части заполнения пропусков). 
При этом диаграмма 2, показывает, чаще всего вакансии без указания Зарплатиной вилки размещают мелкие работодатели, 
в то же время, у крупных работодателей доля вакансий размещенных без зарплатной вилки составляет не более 2 процентов.

Для тех вакансий где указано хотя бы одно значение зарплатной вилки, 
средние показатели составляют от 71065.0 руб. до 110537.0 руб.
Диаграмма 3 показывает, что опыт работы напрямую влияет на уровень, предлагаемой, заработной платы

Наиболее популярными видами рабочего графика при условии по

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [164]:
# текст запроса
query_5_1 = f'''SELECT em.name,
                       Count(vc.id) cnt_vac
                FROM employers em 
                    JOIN
                     vacancies vc ON vc.employer_id = em.id
                GROUP BY em.name
                ORDER BY cnt_vac DESC
                LIMIT 5
'''

In [165]:
# результат запроса
top5emploer = pd.read_sql_query(query_5_1, connection)
display(top5emploer)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\3700526276.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,cnt_vac
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


In [166]:
query_5_1_1 = f'''SELECT em.name,
                         coalesce(vc.salary_from, vc.salary_to)+
                         coalesce(vc.salary_to, vc.salary_from)/2 avg_salary
                  FROM employers em 
                      JOIN
                      vacancies vc ON vc.employer_id = em.id
                      JOIN
                      (SELECT em.id,
                            Count(vc.id) cnt_vac
                       FROM employers em 
                           JOIN
                           vacancies vc ON vc.employer_id = em.id
                       GROUP BY em.id
                       ORDER BY cnt_vac DESC
                       LIMIT 10) top_empl ON em.id = top_empl.id
                  WHERE coalesce(vc.salary_from, vc.salary_to,0) > 0
'''  
empl_salary_df = pd.read_sql_query(query_5_1_1, connection)

fig = px.box(
    data_frame=empl_salary_df,
    x='avg_salary',
    y='name',
    title='Диаграмма 4. Уровень зарплат предлагаемых ТОП10 работодателей по количеству вакансий'
)

fig.update_layout(
    xaxis_title = 'Средний уровень зарплаты',
    yaxis_title = 'Работодатель'
)

fig.show()

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\3691206929.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [167]:
# текст запроса
query_5_2 = f'''SELECT ar.name area,
                    Count(em.id) cnt_emploer,
                    Count(vc.id) cnt_vacancies
                FROM areas ar 
                    LEFT JOIN
                     employers em ON ar.id = em.area
                    LEFT JOIN
                     vacancies vc ON vc.area_id = ar.id
                GROUP BY ar.name
                HAVING Count(vc.id) = 0
                ORDER BY cnt_emploer DESC
                LIMIT 2
'''

In [168]:
# результат запроса
area_emploer_df = pd.read_sql_query(query_5_2, connection)
display(area_emploer_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\318124031.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,cnt_emploer,cnt_vacancies
0,Россия,410,0
1,Казахстан,207,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [169]:
# текст запроса
query_5_3 = f'''SELECT em.name,
                    Count(DISTINCT vc.area_id) cnt_area
                FROM employers em
                    JOIN
                     vacancies vc ON vc.employer_id = em.id
                GROUP BY em.name
                ORDER BY cnt_area DESC  
                LIMIT 1
'''

In [170]:
# результат запроса
max_area_employers_df = pd.read_sql_query(query_5_3, connection)
display(max_area_employers_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\2579675479.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,cnt_area
0,Яндекс,181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [171]:
# текст запроса
query_5_4 = f'''SELECT Count(distinct em.id) cnt_em
                FROM employers em
                    LEFT JOIN
                     employers_industries ei ON ei.employer_id = em.id
                WHERE ei.industry_id is Null
'''

In [172]:
# результат запроса
empl_none_ind_df = pd.read_sql_query(query_5_4, connection)
display(empl_none_ind_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\523700377.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_em
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [173]:
# текст запроса
query_5_5 = f'''SELECT em.name,
                     Count(distinct ei.industry_id) cnt_industry
                FROM employers em
                    LEFT JOIN
                     employers_industries ei ON ei.employer_id = em.id
                GROUP BY em.name
                HAVING Count(distinct ei.industry_id) = 4
                ORDER BY 1
                LIMIT 3
'''

In [174]:
# результат запроса
employ4industry_df = pd.read_sql_query(query_5_5, connection)
display(employ4industry_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\3407114967.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,cnt_industry
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [183]:
# текст запроса
query_5_6 = f'''SELECT Count(DISTINCT em.id) cnt_em
                FROM employers em
                    JOIN
                     employers_industries ei ON ei.employer_id = em.id
                    JOIN
                     industries ind ON ind.id = ei.industry_id
                WHERE ind.name = 'Разработка программного обеспечения'
'''

In [184]:
# результат запроса
soft_employ_df = pd.read_sql_query(query_5_6, connection)

display(soft_employ_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\199136653.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_em
0,3553


In [177]:
query_5_6_1 = f'''SELECT em.name employer,
                         avg(coalesce(vc.salary_from, vc.salary_to)+
                             coalesce(vc.salary_to, vc.salary_from)/2) avg_salary
                  FROM employers em
                      JOIN
                       employers_industries ei ON ei.employer_id = em.id
                      JOIN
                       industries ind ON ind.id = ei.industry_id
                      JOIN
                       vacancies vc ON vc.employer_id = em.id
                  WHERE ind.name = 'Разработка программного обеспечения'
                     AND coalesce(vc.salary_from, vc.salary_to,0) > 0
                  GROUP BY em.name
                  ORDER BY avg_salary DESC       
                  LIMIT 10
'''
soft_employ_df = pd.read_sql_query(query_5_6_1, connection)

fig = px.bar(
    data_frame=soft_employ_df,
    x='employer',
    y='avg_salary',
    color='employer',
    title='Диаграмма 5. ТОП 10 по уровню зарплаты, разработчиков программного обеспечения'
)

fig.update_layout(
    xaxis_title='Работодатель',
    yaxis_title='Уровень зарплаты'
)


C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\4059743229.py:17: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [178]:
# код для получения списка городов-милионников
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url)
page = BeautifulSoup(response.text,'html.parser')
tables = pd.read_html(str(page.find('table')), flavor='lxml', encoding='utf-8')
df_million = tables[0]

list_million = list(df_million['Город'])

where_city = '\'' + '\',\''.join(list_million)  + '\''
display(where_city)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\1765697868.py:9: FutureWarning:

Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



"'Москва','Санкт-Петербург','Новосибирск','Екатеринбург','Казань','Красноярск','Нижний Новгород','Челябинск','Уфа','Краснодар','Самара','Ростов-на-Дону','Омск','Воронеж','Пермь','Волгоград'"

In [179]:
# текст запроса
query_5_7 = f'''SELECT ar.name area,
                       Count(vc.id) cnt_vac
                FROM employers em
                  JOIN
                     vacancies vc ON em.id = vc.employer_id
                  JOIN 
                     areas ar ON vc.area_id = ar.id
                WHERE em.name = 'Яндекс' AND ar.name in({where_city})
                GROUP BY ar.name 
                UNION
                SELECT 'Total' area,
                       Count(vc.id) cnt_vac
                FROM employers em
                  JOIN
                     vacancies vc ON em.id = vc.employer_id
                  JOIN 
                     areas ar ON vc.area_id = ar.id
                WHERE em.name = 'Яндекс' AND ar.name in({where_city})
                ORDER BY cnt_vac
'''

In [180]:
# результат запроса
yandex_df = pd.read_sql_query(query_5_7, connection)
display(yandex_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\2007254004.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,cnt_vac
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

In [181]:
query_5_7_1 = f'''SELECT ar.name area,
                         Coalesce(vc.salary_from, vc.salary_to)+
                         Coalesce(vc.salary_to, vc.salary_from)/2  avg_salary
                  FROM employers em
                    JOIN
                      vacancies vc ON em.id = vc.employer_id
                    JOIN 
                      areas ar ON vc.area_id = ar.id
                  WHERE em.name = 'Яндекс' 
                    AND ar.name in({where_city})
                    AND Coalesce(vc.salary_from, vc.salary_to,0) > 0
'''

yandex_area_salary_df = pd.read_sql_query(query_5_7_1, connection)

fig = px.box(
    data_frame= yandex_area_salary_df,
    x='avg_salary',
    y='area',
    title='Диаграмма 6. Распределение зарплаты предлагаемой Яндекс в городах миллионниках'
)

fig.update_layout(
    xaxis_title='Уровень зарплаты',
    yaxis_title='Город'
)

fig.show()


C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\249127418.py:14: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [186]:
# выводы по анализу работодателей
all_emploers = employers_df['cnt_employers'].iloc[0]

sps_top5employers = '; '.join(list(top5emploer['name']))

non_industry = empl_none_ind_df['cnt_em'].iloc[0]
perc_non_industry = round(non_industry/all_emploers*100)

soft_industry = soft_employ_df['cnt_em'].iloc[0]
perc_soft_industry = round(soft_industry/(all_emploers-non_industry)*100)

res_data = f'''ТОП5 работодателей по количесву выставленных вакансий: {sps_top5employers}.
При этом диаграма 4 показывает, что работодатели размещаемые наибольшее количество вакансий, 
предлагают не самые высокие уровни заработной платы.

{perc_non_industry}% работодателей не указали свою сферу деятельности.

Из тех, что указали свою сферу деятельности, {perc_soft_industry}% занимаются разработкой программного обеспечения.
Работодатели, занимающиеся разработкой программного обеспечения и предлагающие наибольший уровень зарплаты, представлены на диаграме 5.

Компания Яндекс размещает много вакансий в городах миллионниках, но наибольшее количество приходится на Москву и Санкт-Петербург.
Диаграмма 6 показывает, что у Яндекс имеется четкая финансовая политика по уровню предлагаемой зарплаты в регионах, от которой они отходят только в Москве и Санкт-Петербурге
'''


print(res_data)

ТОП5 работодателей по количесву выставленных вакансий: Яндекс; Ростелеком; Тинькофф; СБЕР; Газпром нефть.
При этом диаграма 4 показывает, что работодатели размещаемые наибольшее количество вакансий, 
предлагают не самые высокие уровни заработной платы.

36% работодателей не указали свою сферу деятельности.

Из тех, что указали свою сферу деятельности, 24% занимаются разработкой программного обеспечения.
Работодатели, занимающиеся разработкой программного обеспечения и предлагающие наибольший уровень зарплаты, представлены на диаграме 5.

Компания Яндекс размещает много вакансий в городах миллионниках, но наибольшее количество приходится на Москву и Санкт-Петербург.
Диаграмма 6 показывает, что у Яндекс имеется четкая финансовая политика по уровню предлагаемой зарплаты в регионах, от которой они отходят только в Москве и Санкт-Петербурге



# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса
query_6_1 = f'''SELECT Count(vc.id) cnt_data_vac
                FROM vacancies vc 
                WHERE lower(vc.name) like '%data%' 
                    OR lower(vc.name) like '%данн%'
'''

In [ ]:
# результат запроса
data_vac_df = pd.read_sql_query(query_6_1, connection)
display(data_vac_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\477650802.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_data_vac
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса
where_DS = f'''lower(vc.name) like '%data scientist%' 
               OR lower(vc.name) like '%data science%'
               OR lower(vc.name) like '%исследователь данных%'
               OR (lower(vc.name) like '%ml%' AND lower(vc.name) not like '%html%')
               OR lower(vc.name) like '%machine learning%'
               OR lower(vc.name) like '%машинн%обучен%'                    
'''

query_6_2 = f'''SELECT Count(vc.id) cnt_data_vac
                FROM vacancies vc 
                WHERE ({where_DS})
                  AND (lower(vc.name) like '%junior%'
                      OR lower(vc.experience) = 'нет опыта'
                      OR lower(vc.employment) = 'стажировка')
'''

In [ ]:
# результат запроса
data_vac_df = pd.read_sql_query(query_6_2, connection)
display(data_vac_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\2591735402.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_data_vac
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
query_6_3 = f'''SELECT Count(vc.id) cnt_vac
                FROM vacancies vc
                WHERE ({where_DS}) 
                   AND (lower(vc.key_skills) like '%sql%' OR lower(vc.key_skills) like '%postgres%')
'''


In [ ]:
# результат запроса
data_vac_df = pd.read_sql_query(query_6_3, connection)
display(data_vac_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\2799030205.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_vac
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
query_6_4 = f'''SELECT Count(vc.id) cnt_vac
                FROM vacancies vc
                WHERE ({where_DS}) 
                   AND lower(vc.key_skills) like '%python%'
'''

In [ ]:
# результат запроса
data_vac_df = pd.read_sql_query(query_6_4, connection)
display(data_vac_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\3298470028.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt_vac
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса
query_6_5 = f'''SELECT  round(
                             avg(length(vc.key_skills)-
                                 length(replace(vc.key_skills,'\t',''))+1
                                 )
                              ,2) avg_skills
                FROM vacancies vc
                WHERE ({where_DS}) 
                 AND vc.key_skills is not null
'''

In [ ]:
# результат запроса
data_vac_df = pd.read_sql_query(query_6_5, connection)
display(data_vac_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\2668455232.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,avg_skills
0,6.55


In [ ]:
#для проверки 

query_6_5_dop = f'''SELECT  vc.id,
                        vc.key_skills
                FROM vacancies vc
                WHERE ({where_DS}) 
                 AND vc.key_skills is not null
'''
data_vac_df = pd.read_sql_query(query_6_5_dop, connection)

data_vac_df['cnt_key_skills'] = data_vac_df['key_skills'].apply(lambda x: len(x.split('\t')) if not x is None else 0)

avg_skills = data_vac_df['cnt_key_skills'].sum()/data_vac_df['cnt_key_skills'].count()

display(round(avg_skills,2))

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\1158148765.py:9: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



np.float64(6.55)

Среднее количество указываемых ключевых навыков в вакансиях, у меня получилось 6.55, для самопроверки расчет проводился двумя способами, в запросе SQl (согласно подсказкам) и средствами python на основе общих данных загруженных DataFrame. Почему этот ответ не правильный, я не понимаю, возможно что-то не понял в условиях.

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
query_6_6 = f'''SELECT vc.experience,
                      Round(
                            avg(
                                (coalesce(vc.salary_from,vc.salary_to)+ 
                                 coalesce(vc.salary_to,vc.salary_from))/2
                                )
                            ,0)    avg_salary 
                FROM vacancies vc
                WHERE ({where_DS}) 
                  AND coalesce(vc.salary_from,vc.salary_to,0) > 0
                GROUP BY vc.experience
                ORDER BY avg_salary DESC
'''

In [ ]:
# результат запроса
avg_salary_df = pd.read_sql_query(query_6_6, connection)
display(avg_salary_df)

C:\Users\Владимир\AppData\Local\Temp\ipykernel_51180\2750549478.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,avg_salary
0,От 3 до 6 лет,256454.0
1,Более 6 лет,157933.0
2,От 1 года до 3 лет,144233.0
3,Нет опыта,74643.0


***

In [ ]:
# выводы по предметному анализу

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования